In [1]:
import tensorflow as tf

In [2]:
D = 256

In [3]:
with tf.name_scope("constant"):
    t = tf.placeholder(tf.float64, shape=())
    H = tf.reshape(tf.constant([[0.25,0,0,0],[0,-0.25,0.5,0],[0,0.5,-0.25,0],[0,0,0,0.25]],dtype=tf.float64),[2,2,2,2],name="Hamiltonian")
    I = tf.reshape(tf.constant([[1.,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],dtype=tf.float64),[2,2,2,2],name="Identity")
    expH = I + t * H
    Z = tf.zeros(D,name="Zeros",dtype=tf.float64)

In [4]:
with tf.name_scope("variables"):
    A = tf.Variable(tf.random_uniform([D, D, 2], minval=-1., maxval=1.,dtype=tf.float64),name="A")
    B = tf.Variable(tf.random_uniform([D, D, 2], minval=-1., maxval=1.,dtype=tf.float64),name="B")
    EAB = tf.Variable(tf.ones(D,dtype=tf.float64), name="EAB")
    EBA = tf.Variable(tf.ones(D,dtype=tf.float64), name="EBA")
    L = tf.Variable(tf.random_uniform([D, D], minval=-1., maxval=1.,dtype=tf.float64),name="L")
    R = tf.Variable(tf.random_uniform([D, D], minval=-1., maxval=1.,dtype=tf.float64),name="R")
    init = tf.global_variables_initializer()

In [5]:
def update(A,B,EAB,EBA,s):
    label = "%s_H_%s"%(s[0],s[1])
    with tf.name_scope(label):
        EBA_A = tf.multiply(tf.reshape(EBA,[D,1,1]),A,name="%s_%s"%(s[3],s[0]))
        EBA_A_EAB = tf.multiply(EBA_A,tf.reshape(EAB*EAB,[1,D,1]),name="%s_%s_%s"%(s[3],s[0],s[2]))
        EBA_A_EAB_B = tf.tensordot(EBA_A_EAB,B,[[1],[0]],name="%s_%s_%s_%s"%(s[3],s[0],s[2],s[1]))
        EBA_A_EAB_B_EBA = tf.multiply(EBA_A_EAB_B,tf.reshape(EBA,[1,1,D,1]))
        BIG = tf.transpose(tf.tensordot(EBA_A_EAB_B_EBA,expH,[[1,3],[0,1]]),[0,2,1,3],name=label)
        S, U, V = tf.svd(tf.reshape(BIG,[2*D,2*D]),name="%s_SVD"%label)
        nEAB = tf.sqrt(tf.slice(S,[0],[D]),name="n%s"%s[2]) #s[:D]
        pre_reci = tf.reciprocal(EBA,name="pre_reci") # D
        reci= pre_reci
        #reci = tf.where(tf.is_inf(pre_reci),Z,pre_reci,name="%s_Eviron_Reci"%label) # D
        nA = tf.transpose(tf.multiply(tf.reshape(U[:,:D],[D,2,D]),tf.reshape(reci,[D,1,1])),[0,2,1],name="n%s"%s[0])
        nB = tf.transpose(tf.multiply(tf.reshape(V[:,:D],[D,2,D]),tf.reshape(reci,[D,1,1])),[2,0,1],name="n%s"%s[1])
        return tf.assign(A,nA/tf.norm(nA)),tf.assign(B,nB/tf.norm(nB)),tf.assign(EAB,nEAB/tf.norm(nEAB))

In [6]:
updateAB = update(A,B,EAB,EBA,["A","B","EAB","EBA"])
updateBA = update(B,A,EBA,EAB,["B","A","EBA","EAB"])

In [7]:
with tf.name_scope("updateR"):
    RB = tf.tensordot(tf.tensordot(R,B,[[1],[1]]),B,[[0,2],[1,2]],name="RB")
    RBE = tf.multiply(tf.multiply(RB,tf.reshape(EAB*EAB,[D,1])),tf.reshape(EAB*EAB,[1,D]),name="RBE")
    RBEA = tf.tensordot(tf.tensordot(RBE/tf.norm(RBE),A,[[1],[1]]),A,[[0,2],[1,2]],name="RBEA")
    RBEAE = tf.multiply(tf.multiply(RBEA,tf.reshape(EBA*EBA,[D,1])),tf.reshape(EBA*EBA,[1,D]),name="RBEAE")
    updateR = tf.assign(R,RBEAE/tf.norm(RBEAE))

with tf.name_scope("updateL"):
    LA = tf.tensordot(tf.tensordot(L,A,[[1],[0]]),A,[[0,2],[0,2]],name="LA")
    LAE = tf.multiply(tf.multiply(LA,tf.reshape(EAB*EAB,[D,1])),tf.reshape(EAB*EAB,[1,D]),name="LAE")
    LAEB = tf.tensordot(tf.tensordot(LAE/tf.norm(LAE),B,[[1],[0]]),B,[[0,2],[0,2]],name="LAEB")
    LAEBE = tf.multiply(tf.multiply(LAEB,tf.reshape(EBA*EBA,[D,1])),tf.reshape(EBA*EBA,[1,D]),name="LAEBE")
    updateL = tf.assign(L,LAEBE/tf.norm(LAEBE))

In [8]:
with tf.name_scope("Energy"):
    LA = tf.tensordot(L,A,[[0],[0]])
    LAE = tf.multiply(LA,tf.reshape(EAB*EAB,[1,D,1]))
    LAEB = tf.tensordot(LAE,B,[[1],[0]])
    LAEBR = tf.tensordot(LAEB,R,[[2],[0]])
    LAEBRA = tf.tensordot(LAEBR,A,[[0],[0]])
    LAEBRAE = tf.multiply(LAEBRA,tf.reshape(EAB*EAB,[1,1,1,D,1]))
    LAEBRAEB = tf.tensordot(LAEBRAE,B,[[3,2],[0,1]])
    E_AB = tf.tensordot(LAEBRAEB,H,[[0,1,2,3],[0,1,2,3]])/tf.tensordot(LAEBRAEB,I,[[0,1,2,3],[0,1,2,3]])
    
    LAEBE = tf.multiply(LAEB,tf.reshape(EBA*EBA,[1,1,D,1]))
    LAEBEA = tf.tensordot(LAEBE,A,[[2],[0]])
    LAEBEAE = tf.multiply(LAEBEA,tf.reshape(EAB*EAB,[1,1,1,D,1]))
    LAEBEAEA = tf.tensordot(LAEBEAE,A,[[0,1],[0,2]])
    LAEBEAEAE = tf.multiply(LAEBEAEA,tf.reshape(EAB*EAB,[1,1,1,D]))
    LAEBEAEAEB = tf.tensordot(LAEBEAEAE,B,[[3],[0]])
    LAEBEAEAEBE = tf.multiply(LAEBEAEAEB,tf.reshape(EBA*EBA,[1,1,1,D,1]))
    LAEBEAEAEBEA = tf.tensordot(LAEBEAEAEBE,A,[[3],[0]])
    LAEBEAEAEBEAE = tf.multiply(LAEBEAEAEBEA,tf.reshape(EAB*EAB,[1,1,1,1,D,1]))
    LAEBEAEAEBEAEB = tf.tensordot(LAEBEAEAEBEAE,B,[[4],[0]])
    LAEBEAEAEBEAEBB = tf.tensordot(LAEBEAEAEBEAEB,B,[[1,6],[0,2]])
    LAEBEAEAEBEAEBBR = tf.tensordot(LAEBEAEAEBEAEBB,R,[[5,4],[0,1]])
    E_BA = tf.tensordot(LAEBEAEAEBEAEBBR,H,[[0,1,2,3],[0,1,2,3]])/tf.tensordot(LAEBEAEAEBEAEBBR,I,[[0,1,2,3],[0,1,2,3]])
    
    E = (E_AB+E_BA)/2
    
    tf.summary.scalar('E', E)

In [9]:
sess = tf.Session(config=tf.ConfigProto(
    device_count={'CPU': 8, 'GPU': 1},
    log_device_placement=True
    #intra_op_parallelism_threads=8,
    #inter_op_parallelism_threads=8
))
sess.run(init)
merged = tf.summary.merge_all()
logger = tf.summary.FileWriter('/tmp/data/%d'%D,sess.graph)
tt = 0

In [10]:
%%time
ep = 0.1
for i in xrange(1):
    for j in xrange(10):
        sess.run(updateAB,feed_dict={t: -4*ep})
        sess.run(updateBA,feed_dict={t: -4*ep})
        sess.run(updateBA,feed_dict={t: -4*ep})
        sess.run(updateAB,feed_dict={t: -4*ep})
        sess.run((updateL,updateR))
    logger.add_summary(sess.run(merged), tt)
    tt += 1

logger.flush()
print sess.run(E)

-0.474820219515
CPU times: user 7.02 s, sys: 403 ms, total: 7.42 s
Wall time: 7.52 s
